In [ ]:
import random
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import albumentations as A
from typing import List, Dict, Any

### Hàm cho task

In [ ]:
class Visualize:
    @staticmethod
    def visualize(images: List):
        num_images = len(images)
        plt.figure(figsize=(15, 15))

        for i, image in enumerate(images):
            plt.subplot(1, num_images, i + 1)
            plt.axis('off')
            plt.imshow(image)

        plt.show()

In [ ]:
class TransformImage:
    def __init__(self, image_path: str, label_directory: str):
        self.image_path = image_path
        self.directory = os.path.dirname(image_path)
        self.label_directory = label_directory  # Đường dẫn tới thư mục chứa label

    def transform(self, names: List[str], params=None, random_seed=43):
        transformed_images = []
        image = cv2.imread(self.image_path)
        if image is None:
            raise ValueError(f"Image at {self.image_path} could not be read.")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        for name in names:
            transform_class = getattr(A, name)
            random.seed(random_seed)
            transform_func = transform_class(**params)
            augmented = transform_func(image=image)
            augmented_image = augmented['image']

            # Lưu ảnh đã biến đổi và file label
            self.save_transformed_image(augmented_image, name)
            transformed_images.append(augmented_image)

        return transformed_images

    def save_transformed_image(self, transformed_image, transform_name: str):
        base_name = os.path.splitext(os.path.basename(self.image_path))[0]
        new_image_name = f"{base_name}_{transform_name}.jpg"
        new_image_path = os.path.join(self.directory, new_image_name)

        # Lưu ảnh đã biến đổi
        cv2.imwrite(new_image_path, cv2.cvtColor(transformed_image, cv2.COLOR_RGB2BGR))
        print(f"Saved transformed image as '{new_image_path}'")
        self.create_label_file(base_name, transform_name)

    def create_label_file(self, base_name: str, transform_name: str):
            new_label_name = f"{base_name}_{transform_name}.txt"
            new_label_path = os.path.join(self.label_directory, new_label_name)

            original_label_name = f"{base_name}.txt"
            original_label_path = os.path.join(self.label_directory, original_label_name)

            if os.path.exists(original_label_path):
                with open(original_label_path, 'r') as original_label_file:
                    content = original_label_file.read()  # Đọc nội dung từ file gốc
            else:
                content = f"Label for transformed image {new_label_name}"

            with open(new_label_path, 'w') as label_file:
                label_file.write(content)

            print(f"Created label file as '{new_label_path}'")

In [ ]:
class FileProcessor:
    def __init__(self, image_directory: str, label_directory: str):
        self.image_directory = image_directory
        self.label_directory = label_directory

    def process_images(self, transform_names: List[str], params: Dict[str, Any], random_seed: int = 43):
        for filename in os.listdir(self.image_directory):
            if filename.endswith('.jpg'):
                image_path = os.path.join(self.image_directory, filename)
                model = TransformImage(image_path, self.label_directory)
                transformed_images = model.transform(transform_names, params=params, random_seed=random_seed)  # Biến đổi ảnh



In [ ]:

# Ví dụ sử dụng
image_directory = r'data/train/images'
label_directory = r'data/train/labels'
processor = FileProcessor(image_directory, label_directory)

params = {'p' : 0.7}

transform_name = ['GaussianBlur','RandomBrightnessContrast']

processor.process_images(transform_name, params)

In [ ]:
import os
from PIL import Image

input_folder = image_directory

def convert_png_to_jpg(png_filename):
    png_path = os.path.join(input_folder, png_filename)
    jpg_filename = png_filename.replace(".png", ".jpg")
    jpg_path = os.path.join(input_folder, jpg_filename)

    png_image = Image.open(png_path)
    jpg_image = png_image.convert("RGB")
    jpg_image.save(jpg_path, "JPEG")
    print(f"Đã chuyển đổi: {png_filename} -> {jpg_filename}")
    os.remove(png_path)
    print(f"Đã xóa tệp PNG cũ: {png_filename}")

png_files = [f for f in os.listdir(input_folder) if f.endswith(".png")]


